### Collaborative Filtering avec Suprise

In [1]:
import pandas as pd
from surprise import Dataset, Reader, SVD

In [9]:
# Fonction de recommandation
def generate_recommendations(ratings_updated, favorite_movies):

    new_user_id = ratings_updated['userId'].max() + 1

    for i in favorite_movies:
        newdata = pd.DataFrame([[new_user_id, i, 5.0]], columns=['userId', 'tmdb_id', 'rating'])
        ratings_updated = pd.concat([ratings_updated, newdata], ignore_index=True)

    reader = Reader(rating_scale=(0.5, 5))
    data = Dataset.load_from_df(ratings_updated[['userId', 'tmdb_id', 'rating']], reader)
    svd = SVD()
    train_set = data.build_full_trainset()
    svd.fit(train_set)

    all_pred = []
    movie_id = []
    for movie in ratings_updated['tmdb_id'].unique().tolist():
        pred = svd.predict(new_user_id, movie)
        all_pred.append(pred.est)
        movie_id.append(pred.iid)

    result = pd.DataFrame(list(zip(movie_id, all_pred)), columns=['tmdb_id', 'predicted_rating'])
    result.sort_values(by='predicted_rating', ascending=False, inplace=True)
    
    return result.sort_values(by='predicted_rating', ascending=False).head(30)

In [10]:
# Importation du fichier
ratings_updated = pd.read_csv('src/Movielens_ratings_updated.csv')

# Liste de films favoris (exemple)
favorite_movies = [13448, 402, 204922]

# Test de la fonction de recommandation
generate_recommendations(ratings_updated, favorite_movies)

,tmdb_id,predicted_rating
154,705,4.897889
120,975,4.890437
1229,961,4.887983
21,11,4.870301
96,15,4.863152
95,935,4.855708
50,637,4.853715
7,278,4.849317
68,629,4.847734
1282,7857,4.836449
